<a href="https://colab.research.google.com/github/HCX1010/CNN/blob/main/RAG%20with%20PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安裝必要包

In [11]:
# @title
!pip install langchain sentence-transformers faiss-cpu openai
!pip install langchain-text-splitters
!pip install triton
!pip install transformers accelerate torch
!pip install PyPDF2
!pip install langchain.document_loaders
!pip install langchain-community
!pip install pymupdf
!pip install langchain_huggingface
!pip install -qU "langchain-chroma>=0.1.2"
!pip install langchain-core==1.2.6
!pip install llama-cpp-python
!pip install langchain-classic
!pip install prompt-template

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement langchain.document_loaders (from versions: none)
ERROR: No matching distribution found for langchain.document_loaders
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: l

# 載入資料+切分成向量

In [31]:
# @title
# Loading PDF Document
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("Virtual_characters.pdf")
PDF_data = loader.load()

# Split the text of PDF into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)

# Using the pre-trained model to embed the text to vector
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                  model_kwargs=model_kwargs)
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain_chroma import Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

# 載入LLM模型作為推論用

In [ ]:
# Load and the inference model
from langchain_core.callbacks import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
model_path = "/content/drive/MyDrive/llama-2-7b-chat.Q4_0.gguf"

LLM = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=100,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

# Give the promt to LLM model
from langchain_classic.chains import LLMChain
from langchain_classic.chains.prompt_selector import ConditionalPromptSelector
from langchain_core.prompts import PromptTemplate

Prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""<<SYS>> You are a professional reader. Use the following pieces of context to answer the user's question accurately. If you don't know the answer, just say that you don't know, don't try to make up an answer. <</SYS>>
[INST] Context: {context}
Question: {question}[/INST]
"""
)

# Set the inference model
from langchain_classic.chains import RetrievalQA
retriever = vectordb.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=LLM,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={"prompt": Prompt}
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! callback_manager is not default parameter.
                callback_manager was transferred to model_kwargs.
                Please confirm that callback_manager is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/drive/MyDrive/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: 

# 測驗

In [ ]:
# Question & Answer
query = "Tell me about Alison Hawk's career and age"
qa.invoke(query)